In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

class DataAnalyzerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Data Analyzer")

        # Frame para contener los widgets
        self.frame = tk.Frame(self.root)
        self.frame.pack(padx=20, pady=20)

        # Botón para cargar conjunto de datos
        self.load_button = tk.Button(self.frame, text="Cargar Datos", command=self.load_data)
        self.load_button.pack(pady=10)

        # Etiqueta para mostrar el estado del análisis
        self.analysis_status = tk.Label(self.frame, text="")
        self.analysis_status.pack()

        # Opciones de análisis
        self.analysis_label = tk.Label(self.frame, text="Opciones de Análisis:")
        self.analysis_label.pack()

        self.analysis_options = [
            "Análisis de Estadísticas Descriptivas",
            "Análisis de Regresión Lineal",
            "Análisis de Clasificación",
            "Análisis de Correlación",
            "Análisis de Componentes Principales (PCA)",
            "Análisis de Clustering"
        ]

        self.selected_analysis = tk.StringVar()
        self.selected_analysis.set(self.analysis_options[0])

        self.analysis_dropdown = tk.OptionMenu(self.frame, self.selected_analysis, *self.analysis_options)
        self.analysis_dropdown.pack(pady=5)

        # Botón de confirmación para iniciar el análisis
        self.confirm_button = tk.Button(self.frame, text="Aceptar", command=self.confirm_analysis)
        self.confirm_button.pack(pady=5)
        self.confirm_button.config(state=tk.DISABLED)  # Inicialmente desactivado

    def load_data(self):
        file_path = filedialog.askopenfilename(title="Seleccionar archivo de datos",
                                                filetypes=(("Archivos CSV", "*.csv"), ("Todos los archivos", "*.*")))
        if file_path:
            try:
                # Cargar el conjunto de datos
                self.data = pd.read_csv(file_path)
                print("Conjunto de datos cargado:", file_path)
                self.analysis_status.config(text="Conjunto de datos cargado correctamente.")
                self.confirm_button.config(state=tk.NORMAL)  # Habilitar el botón de confirmación
            except Exception as e:
                messagebox.showerror("Error", f"No se pudo cargar el archivo: {str(e)}")

    def confirm_analysis(self):
        # Realizar análisis según la opción seleccionada
        if self.selected_analysis.get() == "Análisis de Estadísticas Descriptivas":
            self.descriptive_statistics()
        elif self.selected_analysis.get() == "Análisis de Regresión Lineal":
            self.linear_regression_analysis()
        elif self.selected_analysis.get() == "Análisis de Clasificación":
            self.classification_analysis()
        elif self.selected_analysis.get() == "Análisis de Correlación":
            self.correlation_analysis()
        elif self.selected_analysis.get() == "Análisis de Componentes Principales (PCA)":
            self.pca_analysis()
        elif self.selected_analysis.get() == "Análisis de Clustering":
            self.clustering_analysis()

    def descriptive_statistics(self):
        try:
            # Seleccionar columnas relevantes para el análisis
            relevant_columns = ['Total cantidad', 'Total venta', 'Costo total salida', 'Utilidad bruta']
            relevant_data = self.data[relevant_columns]

            # Calcular estadísticas descriptivas
            statistics = relevant_data.describe()
            print("Estadísticas descriptivas del conjunto de datos:")
            print(statistics)
            self.analysis_status.config(text="Análisis de estadísticas descriptivas completado.")
        except KeyError as e:
            messagebox.showerror("Error", "Las columnas necesarias para el análisis no están presentes en el conjunto de datos.")

    def linear_regression_analysis(self):
        try:
            # Seleccionar características y objetivo
            features = self.data[['Total cantidad']]
            target = self.data['Total venta']

            # Convertir la columna objetivo a valores numéricos y manejar valores faltantes
            target = pd.to_numeric(target, errors='coerce')
            features = features.dropna()
            target = target.dropna()

            # Asegurar que ambos tengan el mismo número de muestras
            min_samples = min(len(features), len(target))
            features = features[:min_samples]
            target = target[:min_samples]

            # Dividir datos en conjuntos de entrenamiento y prueba
            if min_samples > 0:
                X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

                # Crear modelo de regresión lineal
                model = LinearRegression()
                model.fit(X_train, y_train)

                # Imprimir coeficientes
                print("Coeficiente de la regresión lineal:", model.coef_)
                self.analysis_status.config(text="Análisis de regresión lineal completado.")
            else:
                messagebox.showerror("Error", "No hay suficientes muestras para realizar el análisis.")
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo realizar el análisis de regresión lineal: {str(e)}")

    def classification_analysis(self):
        try:
            # Seleccionar características y objetivo
            features = self.data[['Total cantidad']]
            target = self.data['Tipo']

            # Convertir la columna objetivo a valores numéricos y manejar valores faltantes
            target = pd.to_numeric(target, errors='coerce')
            features = features.dropna()
            target = target.dropna()

            # Asegurar que ambos tengan el mismo número de muestras
            min_samples = min(len(features), len(target))
            features = features[:min_samples]
            target = target[:min_samples]

            # Dividir datos en conjuntos de entrenamiento y prueba
            if min_samples > 0:
                X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

                # Crear modelo de clasificación
                model = RandomForestClassifier()
                model.fit(X_train, y_train)

                # Predecir en el conjunto de prueba
                predictions = model.predict(X_test)

                # Calcular precisión
                accuracy = accuracy_score(y_test, predictions)
                print("Precisión del modelo de clasificación:", accuracy)
                self.analysis_status.config(text="Análisis de clasificación completado.")
            else:
                messagebox.showerror("Error", "No hay suficientes muestras para realizar el análisis.")
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo realizar el análisis de clasificación: {str(e)}")

    def correlation_analysis(self):
        try:
            # Calcular matriz de correlación
            correlation_matrix = self.data.corr()
            print("Matriz de correlación:")
            print(correlation_matrix)

            # Visualizar la matriz de correlación
            plt.figure(figsize=(10, 8))
            sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
            plt.title("Matriz de Correlación")
            plt.show()
            self.analysis_status.config(text="Análisis de correlación completado.")
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo realizar el análisis de correlación: {str(e)}")

    def pca_analysis(self):
        try:
            # Eliminar filas con valores faltantes
            numeric_data = self.data.select_dtypes(include=['number']).dropna()

            # Aplicar PCA
            pca = PCA(n_components=2)
            principal_components = pca.fit_transform(numeric_data)

            # Visualizar los componentes principales
            plt.figure(figsize=(8, 6))
            plt.scatter(principal_components[:, 0], principal_components[:, 1])
            plt.xlabel('Componente Principal 1')
            plt.ylabel('Componente Principal 2')
            plt.title('Análisis de Componentes Principales (PCA)')
            plt.show()
            self.analysis_status.config(text="Análisis de Componentes Principales (PCA) completado.")
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo realizar el análisis de PCA: {str(e)}")

    def clustering_analysis(self):
        try:
            # Eliminar filas con valores faltantes
            numeric_data = self.data.select_dtypes(include=['number']).dropna()

            # Aplicar algoritmo de clustering (KMeans)
            kmeans = KMeans(n_clusters=3, random_state=42)
            clusters = kmeans.fit_predict(numeric_data)

            # Visualizar los clusters
            plt.figure(figsize=(8, 6))
            plt.scatter(numeric_data.iloc[:, 0], numeric_data.iloc[:, 1], c=clusters, cmap='viridis')
            plt.xlabel('Total cantidad')
            plt.ylabel('Total venta')
            plt.title('Análisis de Clustering')
            plt.show()
            self.analysis_status.config(text="Análisis de Clustering completado.")
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo realizar el análisis de Clustering: {str(e)}")

def main():
    root = tk.Tk()
    app = DataAnalyzerApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
